This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).
This is the basic model and with 250 epochs has an accuracy of 80% (really poor).
I'll continue for a few days researching how much I can optimize this model.

Now switching to using forests in this new kernel https://www.kaggle.com/mulargui/xgboost
You can find all my notes and versions at https://github.com/mulargui/kaggle-Classify-forest-types

In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#load data
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

In [30]:
####### FEATURE ENGINEERING #####
#taking most of this from my work in feature engineering at https://www.kaggle.com/mulargui/xgboost
#https://www.kaggle.com/arateris/2-layer-k-fold-learning-forest-cover 
#Fixing Hillshade_3pm
#replacing the zeros for better guess, mainly to avoid zeros in the feature engineering and fake outliers. 
num_train = len(dftrain)
tmp = dftrain.drop('Cover_Type', axis = 1)
all_data = tmp.append(dftest)

cols_for_HS = ['Aspect','Slope', 'Hillshade_9am','Hillshade_Noon']
HS_zero = all_data[all_data.Hillshade_3pm==0]
HS_train = all_data[all_data.Hillshade_3pm!=0]

from sklearn.ensemble import RandomForestRegressor
rf_hs = RandomForestRegressor(n_estimators=100).fit(HS_train[cols_for_HS], HS_train.Hillshade_3pm)
out = rf_hs.predict(HS_zero[cols_for_HS]).astype(int)

#I couldn't make this line work, feature not used
#all_data.loc[HS_zero.index,'Hillshade_3pm'] = out
#dftrain['Hillshade_3pm']= all_data.loc[:num_train,'Hillshade_3pm']
#dftest['Hillshade_3pm']= all_data.loc[num_train:,'Hillshade_3pm']

# Add PCA features
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99).fit(all_data)
trans = pca.transform(all_data)

for i in range(trans.shape[1]):
    col_name= 'pca'+str(i+1)
    dftrain[col_name] = trans[:num_train, i]
    dftest[col_name] = trans[num_train:, i]

#https://www.kaggle.com/evimarp/top-6-roosevelt-national-forest-competition
def euclidean(df):
    df['Euclidean_distance_to_hydro'] = (df.Vertical_Distance_To_Hydrology**2 
                                         + df.Horizontal_Distance_To_Hydrology**2)**.5
    return df

dftrain = euclidean(dftrain)
dftest = euclidean(dftest)

from itertools import combinations
def distances(df):
    cols = [
        'Horizontal_Distance_To_Roadways',
        'Horizontal_Distance_To_Fire_Points',
        'Horizontal_Distance_To_Hydrology',
    ]
    df['distance_mean'] = df[cols].mean(axis=1)
    df['distance_sum'] = df[cols].sum(axis=1)
    df['distance_road_fire'] = df[cols[:2]].mean(axis=1)
    df['distance_hydro_fire'] = df[cols[1:]].mean(axis=1)
    df['distance_road_hydro'] = df[[cols[0], cols[2]]].mean(axis=1)
    
    df['distance_sum_road_fire'] = df[cols[:2]].sum(axis=1)
    df['distance_sum_hydro_fire'] = df[cols[1:]].sum(axis=1)
    df['distance_sum_road_hydro'] = df[[cols[0], cols[2]]].sum(axis=1)
    
    df['distance_dif_road_fire'] = df[cols[0]] - df[cols[1]]
    df['distance_dif_hydro_road'] = df[cols[2]] - df[cols[0]]
    df['distance_dif_hydro_fire'] = df[cols[2]] - df[cols[1]]
    
    # Vertical distances measures
    colv = ['Elevation', 'Vertical_Distance_To_Hydrology']
    df['Vertical_dif'] = df[colv[0]] - df[colv[1]]
    df['Vertical_sum'] = df[colv].sum(axis=1)
    
    return df
  
dftrain = distances(dftrain)
dftest = distances(dftest)
    
def shade(df):
    SHADES = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    
    df['shade_noon_diff'] = df['Hillshade_9am'] - df['Hillshade_Noon']
    df['shade_3pm_diff'] = df['Hillshade_Noon'] - df['Hillshade_3pm']
    df['shade_all_diff'] = df['Hillshade_9am'] - df['Hillshade_3pm']
    df['shade_sum'] = df[SHADES].sum(axis=1)
    df['shade_mean'] = df[SHADES].mean(axis=1)
    
    return df

dftrain = shade(dftrain)
dftest = shade(dftest)

def elevation(df):
    df['ElevationHydro'] = df['Elevation'] - 0.25 * df['Euclidean_distance_to_hydro']
    return df

dftrain = elevation(dftrain)
dftest = elevation(dftest)

def elevationV(df):
    df['ElevationV'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    return df

dftrain = elevationV(dftrain)
dftest = elevationV(dftest)

def elevationH(df):
    df['ElevationH'] = df['Elevation'] - 0.19 * df['Horizontal_Distance_To_Hydrology']
    return df

dftrain = elevationH(dftrain)
dftest = elevationH(dftest)

def kernel_features(df):
    df['Elevation2'] = df['Elevation']**2
    df['ElevationLog'] = np.log1p(df['Elevation'])
    return df

dftrain = kernel_features(dftrain)
dftest = kernel_features(dftest)

def degree(df):
    df['Aspect_cos'] = np.cos(np.radians(df.Aspect))
    df['Aspect_sin'] = np.sin(np.radians(df.Aspect))
    #df['Slope_sin'] = np.sin(np.radians(df.Slope))
    df['Aspectcos_Slope'] = df.Slope * df.Aspect_cos
    #df['Aspectsin_Slope'] = df.Slope * df.Aspect_sin
    
    return df

dftrain = degree(dftrain)
dftest = degree(dftest)

from bisect import bisect
cardinals = [i for i in range(45, 361, 90)]
points = ['N', 'E', 'S', 'W']

def cardinal(df):
    df['Cardinal'] = df.Aspect.apply(lambda x: points[bisect(cardinals, x) % 4])
    return df

dftrain = cardinal(dftrain)
dftest = cardinal(dftest)

def cardinal_num(df):
    d = {'N': 0, 'E': 1, 'S': 0, 'W':-1}
    df['Cardinal'] = df.Cardinal.apply(lambda x: d[x])
    return df

dftrain = cardinal_num(dftrain)
dftest = cardinal_num(dftest)

#adding features based on https://douglas-fraser.com/forest_cover_management.pdf pages 21,22
#note: not all climatic and geologic codes have a soil type

def Climatic2(row): 
    if (row['Soil_Type1'] == 1) or (row['Soil_Type2'] == 1) or (row['Soil_Type3'] == 1) or (row['Soil_Type4'] == 1) \
        or (row['Soil_Type5'] == 1) or (row['Soil_Type6'] == 1) :
        return 1 
    return 0

dftrain['Climatic2'] = dftrain.apply (lambda row: Climatic2(row), axis=1)
dftest['Climatic2'] = dftest.apply (lambda row: Climatic2(row), axis=1)

def Climatic3(row): 
    if (row['Soil_Type7'] == 1) or (row['Soil_Type8'] == 1) :
        return 1 
    return 0

dftrain['Climatic3'] = dftrain.apply (lambda row: Climatic3(row), axis=1)
dftest['Climatic3'] = dftest.apply (lambda row: Climatic3(row), axis=1)

def Climatic4(row): 
    if (row['Soil_Type9'] == 1) or (row['Soil_Type10'] == 1) or (row['Soil_Type11'] == 1) or (row['Soil_Type12'] == 1) \
        or (row['Soil_Type13'] == 1) :
        return 1 
    return 0

dftrain['Climatic4'] = dftrain.apply (lambda row: Climatic4(row), axis=1)
dftest['Climatic4'] = dftest.apply (lambda row: Climatic4(row), axis=1)

def Climatic5(row): 
    if (row['Soil_Type14'] == 1) or (row['Soil_Type15'] == 1) :
        return 1 
    return 0

dftrain['Climatic5'] = dftrain.apply (lambda row: Climatic5(row), axis=1)
dftest['Climatic5'] = dftest.apply (lambda row: Climatic5(row), axis=1)

def Climatic6(row): 
    if (row['Soil_Type16'] == 1) or (row['Soil_Type17'] == 1) or (row['Soil_Type18'] == 1) :
        return 1 
    return 0

dftrain['Climatic6'] = dftrain.apply (lambda row: Climatic6(row), axis=1)
dftest['Climatic6'] = dftest.apply (lambda row: Climatic6(row), axis=1)

def Climatic7(row): 
    if (row['Soil_Type19'] == 1) or (row['Soil_Type20'] == 1) or (row['Soil_Type21'] == 1) or (row['Soil_Type22'] == 1) \
        or (row['Soil_Type23'] == 1) or (row['Soil_Type24'] == 1) or (row['Soil_Type25'] == 1) or (row['Soil_Type26'] == 1) \
        or (row['Soil_Type27'] == 1) or (row['Soil_Type28'] == 1) or (row['Soil_Type29'] == 1) or (row['Soil_Type30'] == 1) \
        or (row['Soil_Type31'] == 1) or (row['Soil_Type32'] == 1) or (row['Soil_Type33'] == 1) or (row['Soil_Type34'] == 1) :
        return 1 
    return 0

dftrain['Climatic7'] = dftrain.apply (lambda row: Climatic7(row), axis=1)
dftest['Climatic7'] = dftest.apply (lambda row: Climatic7(row), axis=1)

def Climatic8(row): 
    if (row['Soil_Type35'] == 1) or (row['Soil_Type36'] == 1) or (row['Soil_Type37'] == 1) or (row['Soil_Type38'] == 1) \
        or (row['Soil_Type39'] == 1) or (row['Soil_Type40'] == 1) :
        return 1 
    return 0

dftrain['Climatic8'] = dftrain.apply (lambda row: Climatic8(row), axis=1)
dftest['Climatic8'] = dftest.apply (lambda row: Climatic8(row), axis=1)

def Geologic1(row): 
    if (row['Soil_Type14'] == 1) or (row['Soil_Type15'] == 1) or (row['Soil_Type16'] == 1) or (row['Soil_Type17'] == 1) \
        or (row['Soil_Type19'] == 1) or (row['Soil_Type20'] == 1) or (row['Soil_Type21'] == 1) :
        return 1 
    return 0

dftrain['Geologic1'] = dftrain.apply (lambda row: Geologic1(row), axis=1)
dftest['Geologic1'] = dftest.apply (lambda row: Geologic1(row), axis=1)

def Geologic2(row): 
    if (row['Soil_Type9'] == 1) or (row['Soil_Type22'] == 1) or (row['Soil_Type23'] == 1) :
        return 1 
    return 0

dftrain['Geologic2'] = dftrain.apply (lambda row: Geologic2(row), axis=1)
dftest['Geologic2'] = dftest.apply (lambda row: Geologic2(row), axis=1)

def Geologic5(row): 
    if (row['Soil_Type7'] == 1) or (row['Soil_Type8'] == 1) :
        return 1 
    return 0

dftrain['Geologic5'] = dftrain.apply (lambda row: Geologic5(row), axis=1)
dftest['Geologic5'] = dftest.apply (lambda row: Geologic5(row), axis=1)

def Geologic7(row): 
    if (row['Soil_Type1'] == 1) or (row['Soil_Type2'] == 1) or (row['Soil_Type3'] == 1) or (row['Soil_Type4'] == 1) \
        or (row['Soil_Type5'] == 1) or (row['Soil_Type6'] == 1) or (row['Soil_Type10'] == 1) \
        or (row['Soil_Type11'] == 1) or (row['Soil_Type12'] == 1) or (row['Soil_Type13'] == 1) or (row['Soil_Type18'] == 1) \
        or (row['Soil_Type24'] == 1) or (row['Soil_Type25'] == 1) or (row['Soil_Type26'] == 1) or (row['Soil_Type27'] == 1) \
        or (row['Soil_Type28'] == 1) or (row['Soil_Type29'] == 1) or (row['Soil_Type30'] == 1) or (row['Soil_Type31'] == 1) \
        or (row['Soil_Type32'] == 1) or (row['Soil_Type33'] == 1) or (row['Soil_Type34'] == 1) or (row['Soil_Type35'] == 1) \
        or (row['Soil_Type36'] == 1) or (row['Soil_Type37'] == 1) or (row['Soil_Type38'] == 1) or (row['Soil_Type39'] == 1) \
        or (row['Soil_Type40'] == 1) :
        return 1 
    return 0

dftrain['Geologic7'] = dftrain.apply (lambda row: Geologic7(row), axis=1)
dftest['Geologic7'] = dftest.apply (lambda row: Geologic7(row), axis=1)

In [31]:
####### DATA PREPARATION #####
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

#normalize features
def normalize(feature):
    min=x[feature].min()
    min2=x_predict[feature].min()
    if (min2 < min):
        min=min2

    max=x[feature].max()
    max2=x_predict[feature].max()
    if (max2 > max):
        max=max2
        
    x[feature]=(x[feature]-min)/(max-min)                             
    x_predict[feature]=(x_predict[feature]-min)/(max-min)  
    
    return                                

normalize("Elevation")
normalize("Aspect")
normalize("Slope")
normalize("Horizontal_Distance_To_Hydrology")
normalize("Vertical_Distance_To_Hydrology")
normalize("Horizontal_Distance_To_Roadways")
normalize("Hillshade_9am")
normalize("Hillshade_Noon")
normalize("Hillshade_3pm")
normalize("Horizontal_Distance_To_Fire_Points")
normalize("pca1")
normalize("Euclidean_distance_to_hydro")
normalize("distance_mean")
normalize("distance_sum")
normalize("distance_road_fire")
normalize("distance_hydro_fire")
normalize("distance_road_hydro")
normalize("distance_sum_road_fire")
normalize("distance_sum_hydro_fire")
normalize("distance_sum_road_hydro")
normalize("distance_dif_road_fire")
normalize("distance_dif_hydro_road")
normalize("distance_dif_hydro_fire")
normalize("Vertical_dif")
normalize("Vertical_sum")
normalize("shade_noon_diff")
normalize("shade_3pm_diff")
normalize("shade_all_diff")
normalize("shade_sum")
normalize("shade_mean")
normalize("ElevationHydro")
normalize("ElevationV")
normalize("ElevationH")
normalize("Elevation2")
normalize("ElevationLog")
normalize("Aspect_cos")
normalize("Aspect_sin")
normalize("Aspectcos_Slope")

In [32]:
# convert the label to One Hot Encoding
num_classes = 7

#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, stratify=y)

True
True


In [33]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

num_features = x_train.shape[1]

model = Sequential()
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='Adam',
              metrics=['accuracy'])

#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

# Predict!!
y_predict = model.predict(x_predict)

Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 1s 121us/sample - loss: 0.9482 - acc: 0.6128 - val_loss: 0.7841 - val_acc: 0.6693
Epoch 2/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.7614 - acc: 0.6771 - val_loss: 0.7179 - val_acc: 0.6852
Epoch 3/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.7077 - acc: 0.6954 - val_loss: 0.7062 - val_acc: 0.6954
Epoch 4/250
12096/12096 [==============================] - 1s 103us/sample - loss: 0.6900 - acc: 0.7053 - val_loss: 0.6611 - val_acc: 0.7090
Epoch 5/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.6640 - acc: 0.7150 - val_loss: 0.6519 - val_acc: 0.7183
Epoch 6/250
12096/12096 [==============================] - 1s 100us/sample - loss: 0.6466 - acc: 0.7242 - val_loss: 0.6301 - val_acc: 0.7239
Epoch 7/250
12096/12096 [==============================] - 1s 103us/sample - loss: 0.6277 - acc: 0.7297 - va

12096/12096 [==============================] - 1s 98us/sample - loss: 0.3433 - acc: 0.8568 - val_loss: 0.4223 - val_acc: 0.8366
Epoch 59/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.3441 - acc: 0.8597 - val_loss: 0.4333 - val_acc: 0.8380
Epoch 60/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.3405 - acc: 0.8623 - val_loss: 0.4134 - val_acc: 0.8386
Epoch 61/250
12096/12096 [==============================] - 1s 101us/sample - loss: 0.3305 - acc: 0.8676 - val_loss: 0.4166 - val_acc: 0.8356
Epoch 62/250
12096/12096 [==============================] - 1s 103us/sample - loss: 0.3379 - acc: 0.8614 - val_loss: 0.4125 - val_acc: 0.8390
Epoch 63/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.3245 - acc: 0.8671 - val_loss: 0.4076 - val_acc: 0.8406
Epoch 64/250
12096/12096 [==============================] - 1s 95us/sample - loss: 0.3260 - acc: 0.8646 - val_loss: 0.4118 - val_acc: 0.8466
Epoch 65/250
12096/12096

Epoch 116/250
12096/12096 [==============================] - 1s 101us/sample - loss: 0.2528 - acc: 0.9001 - val_loss: 0.4129 - val_acc: 0.8528
Epoch 117/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.2500 - acc: 0.8971 - val_loss: 0.4071 - val_acc: 0.8657
Epoch 118/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.2502 - acc: 0.8985 - val_loss: 0.4226 - val_acc: 0.8568
Epoch 119/250
12096/12096 [==============================] - 1s 110us/sample - loss: 0.2484 - acc: 0.9018 - val_loss: 0.3969 - val_acc: 0.8578
Epoch 120/250
12096/12096 [==============================] - 1s 111us/sample - loss: 0.2445 - acc: 0.9020 - val_loss: 0.4229 - val_acc: 0.8522
Epoch 121/250
12096/12096 [==============================] - 1s 111us/sample - loss: 0.2446 - acc: 0.9020 - val_loss: 0.4164 - val_acc: 0.8532
Epoch 122/250
12096/12096 [==============================] - 1s 110us/sample - loss: 0.2401 - acc: 0.9033 - val_loss: 0.4157 - val_acc: 0.8558
E

12096/12096 [==============================] - 1s 98us/sample - loss: 0.2016 - acc: 0.9181 - val_loss: 0.4333 - val_acc: 0.8585
Epoch 174/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.1950 - acc: 0.9210 - val_loss: 0.4465 - val_acc: 0.8578
Epoch 175/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.1958 - acc: 0.9193 - val_loss: 0.4339 - val_acc: 0.8611
Epoch 176/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.2005 - acc: 0.9210 - val_loss: 0.4513 - val_acc: 0.8578
Epoch 177/250
12096/12096 [==============================] - 1s 98us/sample - loss: 0.2041 - acc: 0.9203 - val_loss: 0.4531 - val_acc: 0.8552
Epoch 178/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.1907 - acc: 0.9242 - val_loss: 0.4423 - val_acc: 0.8585
Epoch 179/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.1919 - acc: 0.9215 - val_loss: 0.4419 - val_acc: 0.8694
Epoch 180/250
12096/

Epoch 231/250
12096/12096 [==============================] - 1s 100us/sample - loss: 0.1689 - acc: 0.9314 - val_loss: 0.4661 - val_acc: 0.8707
Epoch 232/250
12096/12096 [==============================] - 1s 99us/sample - loss: 0.1672 - acc: 0.9385 - val_loss: 0.4668 - val_acc: 0.8667
Epoch 233/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.1635 - acc: 0.9332 - val_loss: 0.4700 - val_acc: 0.8684
Epoch 234/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.1677 - acc: 0.9348 - val_loss: 0.4859 - val_acc: 0.8661
Epoch 235/250
12096/12096 [==============================] - 1s 96us/sample - loss: 0.1719 - acc: 0.9324 - val_loss: 0.4896 - val_acc: 0.8667
Epoch 236/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.1668 - acc: 0.9325 - val_loss: 0.4805 - val_acc: 0.8684
Epoch 237/250
12096/12096 [==============================] - 1s 97us/sample - loss: 0.1671 - acc: 0.9346 - val_loss: 0.4645 - val_acc: 0.8647
Epoch

In [34]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

[3.2264892e-02 9.6772856e-01 1.2379054e-13 3.2390021e-15 6.6117918e-06
 2.5086948e-34 0.0000000e+00] 2
[9.5439595e-01 4.5600817e-02 1.6316273e-12 2.8679335e-17 3.2157147e-06
 2.0724690e-38 0.0000000e+00] 1
[9.3185419e-01 6.8143897e-02 3.3286869e-12 4.3126307e-16 1.8717188e-06
 4.5127945e-38 8.4102675e-36] 1
[8.83472085e-01 1.16524093e-01 5.28255217e-12 1.54925301e-15
 3.81575637e-06 9.50608592e-38 1.02279605e-36] 1
[8.2470053e-01 1.7529199e-01 7.7335126e-12 4.0034034e-15 7.5105409e-06
 2.0462063e-37 1.6086363e-37] 1
[9.6126401e-01 3.8733765e-02 1.8594516e-12 3.2274433e-16 2.1702303e-06
 1.6154307e-38 2.4310878e-38] 1
[9.7336459e-01 2.6633805e-02 1.3550063e-12 9.6200219e-17 1.6466641e-06
 0.0000000e+00 5.2041282e-38] 1
[9.7652721e-01 2.3471422e-02 1.0288696e-12 4.9182710e-17 1.4202045e-06
 0.0000000e+00 5.4119402e-38] 1
[9.3059081e-01 6.9406077e-02 2.5706438e-12 1.4416841e-16 3.0576716e-06
 0.0000000e+00 2.8563250e-38] 1
[9.5264977e-01 4.7347430e-02 1.7913663e-12 7.7697234e-17 2.8940665

In [35]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv